<a href="https://colab.research.google.com/github/CalvHobbes/ai-agents/blob/main/Agentic_RAG_with_LLamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## REF - https://learn.deeplearning.ai/courses/building-agentic-rag-with-llamaindex/lesson/2/router-query-engine

In [1]:
# Download the file dynamically from GitHub
!wget -O agentic_rag_llamaindex_requirements.txt https://raw.githubusercontent.com/CalvHobbes/ai-agents/main/agentic_rag_llamaindex_requirements.txt




--2025-01-22 11:14:27--  https://raw.githubusercontent.com/CalvHobbes/ai-agents/main/agentic_rag_llamaindex_requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252 [text/plain]
Saving to: ‘agentic_rag_llamaindex_requirements.txt’

agentic_rag_llamain 100%[===================>]     252  --.-KB/s    in 0s      

2025-01-22 11:14:28 (3.74 MB/s) - ‘agentic_rag_llamaindex_requirements.txt’ saved [252/252]



In [ ]:
# Install the requirements
!pip install -r agentic_rag_llamaindex_requirements.txt

In [3]:
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

--2025-01-22 11:27:47--  https://openreview.net/pdf?id=VtmBAGCN7o
Resolving openreview.net (openreview.net)... 35.184.86.251
Connecting to openreview.net (openreview.net)|35.184.86.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16911937 (16M) [application/pdf]
Saving to: ‘metagpt.pdf’

metagpt.pdf         100%[===================>]  16.13M  45.9MB/s    in 0.4s    

2025-01-22 11:27:47 (45.9 MB/s) - ‘metagpt.pdf’ saved [16911937/16911937]



In [7]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)


In [94]:
import nest_asyncio

nest_asyncio.apply()

In [8]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAPI_KEY')

In [9]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [51]:
model = Settings.llm

In [96]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [97]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [14]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

In [16]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [17]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to MetaGPT..
The document introduces MetaGPT, a meta-programming framework that enhances multi-agent systems based on Large Language Models (LLMs) through role specialization, workflow management, and efficient communication mechanisms. It incorporates an executable feedback mechanism to improve code generation quality during runtime and achieves state-of-the-art performance on various benchmarks. The document also provides insights into the development process of a software application called the "Drawing App" using MetaGPT, highlighting stages like requirement gathering, UI design, system architecture, implementation approach, testing, and task allocation. It discusses the use of Python libraries, creation of a user-friendly GUI, code generation, unit testing, and performance evaluation. Additionally, it touches upon challenges, ethical concerns, and the impact of MetaGPT o

In [20]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: This choice is more relevant as it focuses on retrieving specific context from the MetaGPT paper, which may contain information on how agents share information with other agents..
Agents share information with other agents by utilizing a shared message pool where they can publish structured messages and subscribe to relevant messages based on their profiles. This shared message pool allows all agents to exchange messages directly, enabling them to access messages from other entities transparently. By storing information in this global message pool, agents can retrieve required information without the need to inquire about other agents and wait for their responses, thus enhancing communication efficiency.


In [21]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: Ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..
The ablation study results show that MetaGPT effectively addresses challenges related to context utilization, code hallucinations, and information overload in the development of recommendation engines. By accurately unfolding natural language descriptions, maintaining information validity, and focusing on granular tasks like requirement analysis, MetaGPT mitigates issues such as ambiguity, incomplete code implementation, and irrelevant information. The use of a global message pool and subscription mechanism helps streamline communication and filter out irrelevant contexts, enhancing the efficiency and utility of the information provided.


In [33]:
response = query_engine.query("summarise ablation study")
print(str(response))

Selecting query engine 1: Ablation study typically involves analyzing specific components or features of a model, which aligns more closely with retrieving specific context from the MetaGPT paper..
An ablation study involves systematically removing or disabling certain components or features of a system to evaluate their individual impact on the overall performance or functionality. This process helps in understanding the importance and contribution of each component towards the system's effectiveness.


In [34]:
response = query_engine.query("who is gabriel cirulli") # name onlt mentioned in an image in the metagpt pdf
print(str(response))

Selecting query engine 1: This choice is more relevant as it focuses on retrieving specific context, which would be helpful in finding information about Gabriel Cirulli..
Gabriel Cirulli is not mentioned in the provided context information.


In [ ]:
## Function Calling ##

In [23]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int:
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)

add_tool = FunctionTool.from_defaults(add)
mystery_tool = FunctionTool.from_defaults(mystery)

In [29]:
Settings.llm.predict_and_call([add_tool, mystery_tool],
    "mystify 9 and 8",
    verbose=True)

=== Calling Function ===
Calling function: mystery with args: {"x": 9, "y": 8}
=== Function Output ===
289


AgentChatResponse(response='289', sources=[ToolOutput(content='289', tool_name='mystery', raw_input={'args': (), 'kwargs': {'x': 9, 'y': 8}}, raw_output=289, is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

In [56]:
len(nodes)

34

In [ ]:
print(nodes[33].get_content(metadata_mode="all"))

In [35]:
from llama_index.core import VectorStoreIndex
vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=4)


In [37]:
response = query_engine.query("What are some high-level results of MetaGPT?")
print(str(response))

MetaGPT outperforms all preceding approaches in both HumanEval and MBPP benchmarks. When MetaGPT collaborates with GPT-4, it significantly improves the Pass @k in the HumanEval benchmark compared to GPT-4. It achieves 85.9% and 87.7% pass rates on the MBPP and HumanEval with a single attempt. Additionally, MetaGPT achieves an average score of 3.9, surpassing ChatDev’s score of 2.1, based on the Chat chain.


In [45]:
from llama_index.core.vector_stores import MetadataFilters
query_engine = vector_index.as_query_engine(similarity_top_k=2,
                                            filters=MetadataFilters.from_dicts(
                                                [
                                                    {"key": "page_label", "value": "3"}

                                                  ])
)


In [46]:
response = query_engine.query(
    "What are some high-level results of MetaGPT?",
)
print(str(response))

MetaGPT demonstrates state-of-the-art performance on HumanEval and MBPP, showcasing its effectiveness as a meta-programming framework for developing LLM-based multi-agent systems. Additionally, MetaGPT integrates human-like Standard Operating Procedures (SOPs) to enhance robustness and reduce unproductive collaboration among LLM-based agents. The framework also introduces an executive feedback mechanism that debugs and executes code during runtime, leading to a significant improvement in code generation quality.


In [47]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '3', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2025-01-22', 'last_modified_date': '2025-01-22'}
{'page_label': '3', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2025-01-22', 'last_modified_date': '2025-01-22'}


In [122]:
from typing import List
from llama_index.core.vector_stores import FilterCondition

def total_page_count() -> int:
  """
  Returns total number of pages in MetaGPT document, Only used to fetch the numnber of pages of the document
  """
  return 29

def vector_query(query:str, page_numbers: List[str]) -> str:
  """Perform a vector search over an index.
  query (str): the string query to be embedded.
  page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
  over all pages. Otherwise, filter by the set of specified pages.
  """
  filters = [
      {"key": "page_label", "value": page} for page in page_numbers
  ]
  query_engine = vector_index.as_query_engine(similarity_top_k=2,
                                            filters=MetadataFilters.from_dicts(
                                                filters,
                                                condition=FilterCondition.OR
                                                  )
                                            )
  response = query_engine.query(query)
  return response

vector_query_tool = FunctionTool.from_defaults( name="vector_query",
    fn=vector_query)
page_count_tool = FunctionTool.from_defaults(name="page_count_tool", fn=total_page_count)

In [ ]:
model.predict_and_call([vector_query_tool], "What are the high-level results of MetaGPT as described on the second page?", verbose=True)

In [98]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

# summary_index = SummaryIndex(nodes)

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)


In [106]:
response = model.predict_and_call(
    [vector_query_tool, summary_tool],
    "What are the MetaGPT comparisons with ChatDev described on second page?",
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "MetaGPT comparisons with ChatDev", "page_numbers": ["2"]}
=== Function Output ===
MetaGPT outperforms ChatDev in handling higher levels of software complexity and offering extensive functionality. In experimental evaluations, MetaGPT achieves a 100% task completion rate, showcasing the robustness and efficiency of its design in terms of time and token costs.


In [75]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '8', 'file_name': 'metagpt.pdf', 'file_path': 'metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2025-01-22', 'last_modified_date': '2025-01-22'}


In [101]:
response = model.predict_and_call(
    [vector_query_tool, summary_tool],
    "what is the summary of the paper?",
    verbose=True
)

=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "summary of the paper"}
=== Function Output ===
The paper introduces MetaGPT, a meta-programming framework that enhances multi-agent systems based on Large Language Models (LLMs) through Standardized Operating Procedures (SOPs). It incorporates role specialization, workflow management, and efficient communication mechanisms to facilitate collaborative software development. MetaGPT involves agents like Product Managers, Architects, Engineers, and QA Engineers, each with specific roles. The framework utilizes structured communication interfaces, a publish-subscribe mechanism, and an executable feedback mechanism to improve code generation quality. Experimental results demonstrate MetaGPT's effectiveness in software development tasks, outperforming existing approaches in various benchmarks. The paper also discusses the development process of a software application called the "Drawing App" using MetaGPT, outl

In [ ]:
for n in response.source_nodes:
    print(n.metadata)

In [115]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [118]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool, page_count_tool],
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [119]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "Agent roles in MetaGPT"}
=== Function Output ===
The roles in MetaGPT include the Product Manager responsible for creating the Product Requirement Document and analyzing user stories and competitive analysis, the Architect who designs technical specifications and system architecture diagrams, the Project Manager who breaks down tasks and assigns them to Engineers, the Engineers who develop code based on specifications, and the QA Engineer who generates unit test code and reviews the code for quality assurance. Each agent has a specific role contributing to the success of software projects in MetaGPT.
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "Communication between agent roles in MetaGPT"}
=== Function Output ===
The communication between agent role

In [ ]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

In [129]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "results over HumanEval dataset in MetaGPT document"}
=== Function Output ===
GPT-4 consistently outperformed GPT-3.5-Turbo in various settings on the HumanEval dataset, achieving higher scores across different experiments.
=== LLM Response ===
GPT-4 consistently outperformed GPT-3.5-Turbo in various settings on the HumanEval dataset, achieving higher scores across different experiments.


In [131]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [132]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "agent roles in MetaGPT document"}
=== Function Output ===
The agent roles in the MetaGPT document include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has distinct responsibilities within the software development process. The Product Manager focuses on creating the Product Requirement Document and conducting competitive analysis. The Architect is responsible for designing the system architecture and technical specifications. The Project Manager breaks down the project into tasks for execution. The Engineer develops the code based on provided specifications, while the QA Engineer creates unit tests and ensures software quality. Each agent contributes uniquely to the success of the project by fulfilling their designated role.
=== Calling Function ==

In [143]:
for key in dict(step_output).keys():
    print(key)

output
task_step
next_steps
is_last


In [146]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 7299ebaf-f69c-4660-a06c-0857a6fe29cb: 1
The agent roles in the MetaGPT document include the Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has distinct responsibilities within the software development process. The Product Manager focuses on creating the Product Requirement Document and conducting competitive analysis. The Architect is responsible for designing the system architecture and technical specifications. The Project Manager breaks down the project into tasks for execution. The Engineer develops the code based on provided specifications, while the QA Engineer creates unit tests and ensures software quality. Each agent contributes uniquely to the success of the project by fulfilling their designated role.


In [147]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 7299ebaf-f69c-4660-a06c-0857a6fe29cb: 1


TaskStep(task_id='7299ebaf-f69c-4660-a06c-0857a6fe29cb', step_id='cae33ec5-5744-41e0-827b-c9db51270729', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [148]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "how agents share information in MetaGPT document"}
=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool and a publish-subscribe mechanism. This allows for direct message exchange among all agents and enables them to subscribe to relevant messages based on their roles. The shared message pool and subscription mechanism enhance communication efficiency by providing a centralized platform for information exchange and ensuring that agents receive only task-related information, thus avoiding information overload.


In [149]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool and a publish-subscribe mechanism. This allows for direct message exchange among all agents and enables them to subscribe to relevant messages based on their roles. The shared message pool and subscription mechanism enhance communication efficiency by providing a centralized platform for information exchange and ensuring that agents receive only task-related information, thus avoiding information overload.
True


In [158]:
response = agent.finalize_response(task.task_id)
print(str(response))

Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool and a publish-subscribe mechanism. This allows for direct message exchange among all agents and enables them to subscribe to relevant messages based on their roles. The shared message pool and subscription mechanism enhance communication efficiency by providing a centralized platform for information exchange and ensuring that agents receive only task-related information, thus avoiding information overload.


## Multi Document RAG##

In [160]:
urls = [
    "https://openreview.net/pdf?id=VtmBAGCN7o",
    "https://openreview.net/pdf?id=6PmJoRfdaK",
    "https://openreview.net/pdf?id=hSyW5go0v8",
]

papers = [
    "metagpt.pdf",
    "longlora.pdf",
    "selfrag.pdf",
]

In [162]:
import requests
for url, filename in zip(urls, papers):
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad responses

    # Save the PDF to the 'pdfs' directory
    filepath = os.path.join("", filename)
    with open(filepath, "wb") as f:
        f.write(response.content)

    print(f"Downloaded {filename} from {url}")

print("All PDFs downloaded successfully!")

Downloaded metagpt.pdf from https://openreview.net/pdf?id=VtmBAGCN7o
Downloaded longlora.pdf from https://openreview.net/pdf?id=6PmJoRfdaK
Downloaded selfrag.pdf from https://openreview.net/pdf?id=hSyW5go0v8
All PDFs downloaded successfully!


In [163]:
from typing import Optional,List
from llama_index.core.vector_stores import FilterCondition
def get_doc_tools(
    file_path: str,
    name: str,
) -> str:
    """Get vector query and summary query tools from a document."""

    # load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)

    def vector_query(
        query: str,
        page_numbers: Optional[List[str]] = None
    ) -> str:
        """Use to answer questions over a given paper.

        Useful if you have specific questions over the paper.
        Always leave page_numbers as None UNLESS there is a specific page you want to search for.

        Args:
            query (str): the string query to be embedded.
            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE
                if we want to perform a vector search
                over all pages. Otherwise, filter by the set of specified pages.

        """

        page_numbers = page_numbers or []
        metadata_dicts = [
            {"key": "page_label", "value": p} for p in page_numbers
        ]

        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts,
                condition=FilterCondition.OR
            )
        )
        response = query_engine.query(query)
        return response


    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}",
        fn=vector_query
    )

    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    summary_tool = QueryEngineTool.from_defaults(
        name=f"summary_tool_{name}",
        query_engine=summary_query_engine,
        description=(
            f"Useful for summarization questions related to {name}"
        ),
    )

    return vector_query_tool, summary_tool